# Checks if Perfsonar data is indexed

This notebook checks number of indexed documents in all four doc types (latency, packetloss, throughput and traceroute) and alerts if any of them is 0. It sends mails to all the people substribed to that alert. It is run every 30 min from a cron job.

In [1]:
import subscribers, alerts
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

## find all the indices to look at

In [2]:
indices = es.cat.indices(index="network_weather-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

cday = datetime.utcnow()
pday = cday - timedelta(days=1)
#ind1 = 'network_weather-%d.%02d.%02d' % (cday.year, cday.month, cday.day)
#ind2 = 'network_weather-%d.%02d.%02d' % (pday.year, pday.month, pday.day)
ind1 = 'network_weather-%d.%d.%d' % (cday.year, cday.month, cday.day)
ind2 = 'network_weather-%d.%d.%d' % (pday.year, pday.month, pday.day)

print('checking for indices:', ind1, ind2)

ind=[]
if ind1 in indices:
    ind.append(ind1)
if ind2 != ind1 and ind2 in indices and cday.hour<9:   # not necessarily 3, just indicate it is the beginning period of new day
    ind.append(ind2)

if len(ind)==0:
    print('no current indices found. Aborting.')
    exit
else:
    print('will use indices:', ind)

print('current time',datetime.utcnow())

checking for indices: network_weather-2017.1.30 network_weather-2017.1.29
will use indices: ['network_weather-2017.1.30', 'network_weather-2017.1.29']
current time 2017-01-30 05:59:44.978492


In [3]:
# There is a time offset here - we do now-9 instead of expected now-1. There are two possible reasons:
# a) timestamps are correct but we simply get data with a long delay
# b) timestamps are wrong.

types_query={
   "size": 0,
   "query": {
        "bool":{
            "filter":{
                "range":{"timestamp":{"gt": "now-9h"}}
            }
        }
   },
    "aggs" : {
        "docType" : {
            "terms" : { "field" : "_type" }
        }
    }
}

res = es.search(index=ind, body=types_query, request_timeout=120)
print(res)

{'took': 165, '_shards': {'failed': 0, 'total': 10, 'successful': 10}, 'hits': {'max_score': 0.0, 'total': 1326280, 'hits': []}, 'aggregations': {'docType': {'sum_other_doc_count': 0, 'buckets': [{'doc_count': 562019, 'key': 'packet_loss_rate'}, {'doc_count': 556704, 'key': 'latency'}, {'doc_count': 197678, 'key': 'traceroute'}, {'doc_count': 9879, 'key': 'throughput'}], 'doc_count_error_upper_bound': 0}}, 'timed_out': False}


In [4]:
problematic=[]

if res['hits']['total']==0:
    problematic=['latency', 'packet_loss_rate', 'traceroute', 'throughput']

r=res['aggregations']['docType']['buckets']
for t in r:
    print(t)
    if t['doc_count']==0:
        problematic.append(t['key'])
print('types with no docs:', problematic)

{'doc_count': 562019, 'key': 'packet_loss_rate'}
{'doc_count': 556704, 'key': 'latency'}
{'doc_count': 197678, 'key': 'traceroute'}
{'doc_count': 9879, 'key': 'throughput'}
types with no docs: []


In [5]:
if len(problematic) > 0:
    S = subscribers.subscribers()
    A = alerts.alerts()

    testName = 'Perfsonar'
    subscribersToPerfsonar =  S.getSubscribers(testName)
    print (subscribersToPerfsonar)
    for subscriber in subscribersToPerfsonar:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that there is an issue in indexing Perfsonar data in UC Elasticsearch.\n'
        body = body + 'This is number of documents indexed in last 30 minutes per document type:\n'
        body = body + str(r)+'\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],str(r))